In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from joblib import Parallel, delayed
from tqdm import tqdm
from scipy.spatial.distance import cdist
import torch
import time

# ----- Helper functions -----

def gaussian_kernel_matrix(X, Y=None, sigma=1.0):
    if Y is None:
        Y = X  # If Y is not provided, compute self-kernel
    pairwise_sq_dists = torch.cdist(X, Y, p=2) ** 2
    return torch.exp(-pairwise_sq_dists / (2 * sigma ** 2))

def compute_kernels(data1, data2, sigma=1.0, device='cuda'):
    data1, data2 = torch.tensor(data1, device=device), torch.tensor(data2, device=device)
    K_XX = gaussian_kernel_matrix(data1, sigma=sigma)
    K_YY = gaussian_kernel_matrix(data2, sigma=sigma)
    K_XY = gaussian_kernel_matrix(data1, data2, sigma=sigma)
    return K_XX, K_YY, K_XY

def mmd_torch(K_XX, K_YY, K_XY):
    K_XX.fill_diagonal_(0)
    K_YY.fill_diagonal_(0)
    K_XY.fill_diagonal_(0)
    n = K_XX.shape[0]
    return (torch.sum(K_XX) + torch.sum(K_YY) - 2 * torch.sum(K_XY)) / (n * (n - 1))


def compute_ell11_ell12(K_XX, K_YY, K_XY):
    n = K_XX.shape[0]
    mid = n // 2
    
    ell_11_values = (torch.sum(K_XX[:mid, :mid], dim=1) + torch.sum(K_YY[:mid, :mid], dim=1) - 2 * torch.sum(K_XY[:mid, :mid], dim=1)) / mid
    ell_12_values = (torch.sum(K_XX[mid:, mid:], dim=1) + torch.sum(K_YY[mid:, mid:], dim=1) - 2 * torch.sum(K_XY[mid:, mid:], dim=1)) / mid    
    return ell_11_values.cpu().numpy(), ell_12_values.cpu().numpy()


def generate_data(mean, cov, n, m):
    data = np.random.multivariate_normal(mean, cov, n)
    Y, X = data[:, :2], data[:, 2:]
    # Split the data into training and testing sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=42)
    X1, Y1 = X_train, Y_train
    X2, Y2 = X_test, Y_test

    data_unlabel = np.random.multivariate_normal(mean, cov, m)
    X_unlabel = data_unlabel[:, 2:]
    # Split the new data into two sets
    X1_unlabel, X2_unlabel = train_test_split(X_unlabel, test_size=0.5, random_state=42)
    
    return X1, Y1, X2, Y2, X1_unlabel, X2_unlabel

def calculate_Uss(Y, f1_label, f2_label, f1_unlabel, f2_unlabel, mmd_value):
    mmd_value_ss = mmd_value - np.mean(np.concatenate((f1_label, f2_label))) \
                         + np.mean(np.concatenate((f1_label, f2_label, f1_unlabel, f2_unlabel)))
    return mmd_value_ss

def calculate_Uss_optimal(Y, f1_label, f2_label, f1_unlabel, f2_unlabel, ell1_Y1, ell1_Y2, mmd_value):
    gamma1 = np.mean((ell1_Y1 - np.mean(ell1_Y1)) * (f1_label - np.mean(f1_label))) \
             / np.var(np.concatenate((f1_label, f1_unlabel)))
    gamma2 = np.mean((ell1_Y2 - np.mean(ell1_Y2)) * (f2_label - np.mean(f2_label))) \
             / np.var(np.concatenate((f2_label, f2_unlabel)))
    mmd_value_ss = mmd_value - np.mean(np.concatenate((gamma1 * f1_label, gamma2 * f2_label))) \
                         + np.mean(np.concatenate((gamma1 * f1_label, gamma2 * f2_label, gamma1 * f1_unlabel, gamma2 * f2_unlabel)))
    return mmd_value_ss

def calculate_Uss_optimal_pooled(Y, f1_label_1, f2_label_1, f1_unlabel_1, f2_unlabel_1, 
                                 f1_label_2, f2_label_2, f1_unlabel_2, f2_unlabel_2, 
                                 f1_label_3, f2_label_3, f1_unlabel_3, f2_unlabel_3, 
                                 ell1_Y1, ell1_Y2, mmd_value):
    var_matrix_1 = np.cov(np.vstack((np.concatenate((f1_label_1, f1_unlabel_1)), np.concatenate((f1_label_2, f1_unlabel_2)), np.concatenate((f1_label_3, f1_unlabel_3)))))
    inv_var_matrix_1 = np.linalg.inv(var_matrix_1)
    var_matrix_2 = np.cov(np.vstack((np.concatenate((f2_label_1, f2_unlabel_1)), np.concatenate((f2_label_2, f2_unlabel_2)), np.concatenate((f2_label_3, f2_unlabel_3)))))
    inv_var_matrix_2 = np.linalg.inv(var_matrix_2)
    cov_matrix_1 = np.array([[np.mean((ell1_Y1 - np.mean(ell1_Y1)) * (f1_label_1 - np.mean(f1_label_1))), 
                              np.mean((ell1_Y1 - np.mean(ell1_Y1)) * (f1_label_2 - np.mean(f1_label_2))),
                              np.mean((ell1_Y1 - np.mean(ell1_Y1)) * (f1_label_3 - np.mean(f1_label_3)))]])
    cov_matrix_2 = np.array([[np.mean((ell1_Y2 - np.mean(ell1_Y2)) * (f2_label_1 - np.mean(f2_label_1))),  
                              np.mean((ell1_Y2 - np.mean(ell1_Y2)) * (f2_label_2 - np.mean(f2_label_2))),
                              np.mean((ell1_Y2 - np.mean(ell1_Y2)) * (f2_label_3 - np.mean(f2_label_3)))]])
    gamma1, gamma2 = inv_var_matrix_1 @ cov_matrix_1.T, inv_var_matrix_2 @ cov_matrix_2.T

    mmd_value_ss = mmd_value \
      - np.mean(np.concatenate((gamma1[0] * f1_label_1, gamma2[0] * f2_label_1))) \
      + np.mean(np.concatenate((gamma1[0] * f1_label_1, gamma2[0] * f2_label_1, gamma1[0] * f1_unlabel_1, gamma2[0] * f2_unlabel_1))) \
      - np.mean(np.concatenate((gamma1[1] * f1_label_2, gamma2[1] * f2_label_2))) \
      + np.mean(np.concatenate((gamma1[1] * f1_label_2, gamma2[1] * f2_label_2, gamma1[1] * f1_unlabel_2, gamma2[1] * f2_unlabel_2))) \
      - np.mean(np.concatenate((gamma1[2] * f1_label_3, gamma2[2] * f2_label_3))) \
      + np.mean(np.concatenate((gamma1[2] * f1_label_3, gamma2[2] * f2_label_3, gamma1[2] * f1_unlabel_3, gamma2[2] * f2_unlabel_3)))
    return mmd_value_ss

def fit_and_predict(X1, Y1, X2, Y2, X1_unlabel, X2_unlabel, modelA, modelB, ell1_Y1, ell1_Y2):
    modelA.fit(X1, ell1_Y1)
    modelB.fit(X2, ell1_Y2)
    f1_label = modelB.predict(X1)
    f1_unlabel = modelB.predict(X1_unlabel)
    f2_label = modelA.predict(X2)
    f2_unlabel = modelA.predict(X2_unlabel)
    return f1_label, f1_unlabel, f2_label, f2_unlabel


# ----- Simulation settings -----

device = 'cuda' if torch.cuda.is_available() else 'cpu'

n = 1000        # number of labeled samples
m = 50000         # number of unlabeled samples
d = 102          # total dimensions: 2 response + 100 predictors
num_iterations = 1000
rho_values = [0, 0.25, 0.5, 0.9]
mu = 2
true = 2/np.sqrt(3) * (1 - np.exp(-2 * mu**2 / 3))

# The overall mean is zero.
mean = np.zeros(d)
mean[0] = mu
mean[1] = -mu

# ----- Loop over different rho values -----

results_summary = {}

for rho in rho_values:
    print(f"\nRunning simulations for rho = {rho} ...")
    
    # Build the covariance matrix:
    # We'll make a block for the first s variables (response and first predictor) with off-diagonals = rho,
    # and the remaining predictors are independent.
    cov = np.full((d, d), rho)
    np.fill_diagonal(cov, 1)  # set all variances to 1

    # Define a version of run_iteration that uses the current cov and mean.
    def run_iteration(_):
        X1, Y1, X2, Y2, X1_unlabel, X2_unlabel = generate_data(mean, cov, n, m)
        Y = np.concatenate((Y1, Y2))

        data1 = Y[:,0]
        data2 = Y[:,1]

        K_XX, K_YY, K_XY = compute_kernels(data1[:, np.newaxis], data2[:, np.newaxis], sigma=1.0, device=device)
        mmd_value = mmd_torch(K_XX, K_YY, K_XY)
        ell1_Y1, ell1_Y2 = compute_ell11_ell12(K_XX, K_YY, K_XY)

        # ----- Random Forest -----
        modelA_rf = RandomForestRegressor(n_estimators=100, random_state=42)
        modelB_rf = RandomForestRegressor(n_estimators=100, random_state=42)
        f1_label_rf, f1_unlabel_rf, f2_label_rf, f2_unlabel_rf = \
            fit_and_predict(X1, Y1, X2, Y2, X1_unlabel, X2_unlabel, modelA_rf, modelB_rf, ell1_Y1, ell1_Y2)
        result_rf = calculate_Uss(Y, f1_label_rf, f2_label_rf, f1_unlabel_rf, f2_unlabel_rf, mmd_value)
        result_rf_optimal = calculate_Uss_optimal(Y, f1_label_rf, f2_label_rf, f1_unlabel_rf, f2_unlabel_rf, 
                                                   ell1_Y1, ell1_Y2, mmd_value)
        
        # ----- XGBoost -----
        modelA_xgb = XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
        modelB_xgb = XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
        f1_label_xgb, f1_unlabel_xgb, f2_label_xgb, f2_unlabel_xgb = \
            fit_and_predict(X1, Y1, X2, Y2, X1_unlabel, X2_unlabel, modelA_xgb, modelB_xgb, ell1_Y1, ell1_Y2)
        result_xgb = calculate_Uss(Y, f1_label_xgb, f2_label_xgb, f1_unlabel_xgb, f2_unlabel_xgb, mmd_value)
        result_xgb_optimal = calculate_Uss_optimal(Y, f1_label_xgb, f2_label_xgb, f1_unlabel_xgb, f2_unlabel_xgb, 
                                                   ell1_Y1, ell1_Y2, mmd_value)
        
        # ---- First Coordinate ----
        f1_label_linear = X1[:, 0]
        f2_label_linear = X2[:, 0]
        f1_unlabel_linear = X1_unlabel[:, 0]
        f2_unlabel_linear = X2_unlabel[:, 0]
        result_linear = calculate_Uss(Y, f1_label_linear, f2_label_linear, f1_unlabel_linear, f2_unlabel_linear, mmd_value)
        result_linear_optimal = calculate_Uss_optimal(Y, f1_label_linear, f2_label_linear, f1_unlabel_linear, f2_unlabel_linear,
                                                        ell1_Y1, ell1_Y2, mmd_value) 
        
        # ----- Optimal Pool -----
        result_optimal_pool = calculate_Uss_optimal_pooled(
            Y, 
            f1_label_rf, f2_label_rf, f1_unlabel_rf, f2_unlabel_rf, 
            f1_label_xgb, f2_label_xgb, f1_unlabel_xgb, f2_unlabel_xgb, 
            f1_label_linear, f2_label_linear, f1_unlabel_linear, f2_unlabel_linear,
            ell1_Y1, ell1_Y2, mmd_value)

        base = np.abs(mmd_value - true) ** 2
        mse_rf = np.abs(result_rf - true) ** 2
        mse_xgb = np.abs(result_xgb - true) ** 2
        mse_linear = np.abs(result_linear - true) ** 2
        mse_rf_optimal = np.abs(result_rf_optimal - true) ** 2
        mse_xgb_optimal = np.abs(result_xgb_optimal - true) ** 2
        mse_linear_optimal = np.abs(result_linear_optimal - true) ** 2
        mse_optimal_pool = np.abs(result_optimal_pool - true) ** 2

        return base, mse_rf, mse_xgb, mse_linear, mse_rf_optimal, mse_xgb_optimal, mse_linear_optimal, mse_optimal_pool

    # Run the iterations in parallel
    results_base = []
    results_rf = []
    results_xgb = []
    results_linear = []
    results_rf_optimal = []
    results_xgb_optimal = []
    results_linear_optimal = []
    results_optimal_pool = []

    results = Parallel(n_jobs=-1)(delayed(run_iteration)(_) for _ in tqdm(range(num_iterations), desc="Iterations"))
    
    for base, mse_rf, mse_xgb, mse_linear, mse_rf_optimal, mse_xgb_optimal, mse_linear_optimal, mse_optimal_pool in results:
        results_base.append(base)
        results_rf.append(mse_rf)
        results_xgb.append(mse_xgb)
        results_linear.append(mse_linear)
        results_rf_optimal.append(mse_rf_optimal)
        results_xgb_optimal.append(mse_xgb_optimal)
        results_linear_optimal.append(mse_linear_optimal)
        results_optimal_pool.append(mse_optimal_pool)

    mean_base = np.mean(results_base)
    mean_rf = np.mean(results_rf)
    mean_xgb = np.mean(results_xgb)
    mean_linear = np.mean(results_linear)
    mean_rf_optimal = np.mean(results_rf_optimal)
    mean_xgb_optimal = np.mean(results_xgb_optimal)
    mean_linear_optimal = np.mean(results_linear_optimal)
    mean_optimal_pool = np.mean(results_optimal_pool)
    
    # Save results for this rho value
    results_summary[rho] = {
        "RF/Base": mean_rf / mean_base,
        "XGB/Base": mean_xgb / mean_base,
        "Linear/Base": mean_linear / mean_base,
        "RF Optimal/Base": mean_rf_optimal / mean_base,
        "XGB Optimal/Base": mean_xgb_optimal / mean_base,
        "Linear Optimal/Base": mean_linear_optimal / mean_base,
        "Optimal Pool/Base": mean_optimal_pool / mean_base,
    }
    
    # Print the results for this rho
    print(f"Results for rho = {rho}:")
    print(f"  RF / Base:         {results_summary[rho]['RF/Base']:.4f}")
    print(f"  XGB / Base:        {results_summary[rho]['XGB/Base']:.4f}")
    print(f"  Linear / Base:      {results_summary[rho]['Linear/Base']:.4f}")
    print(f"  RF Optimal / Base: {results_summary[rho]['RF Optimal/Base']:.4f}")
    print(f"  XGB Optimal / Base:{results_summary[rho]['XGB Optimal/Base']:.4f}")
    print(f"  Linear Optimal / Base:{results_summary[rho]['Linear Optimal/Base']:.4f}")
    print(f"  Optimal Pool / Base:{results_summary[rho]['Optimal Pool/Base']:.4f}")
    

    # Print results for each rho
    for rho in results_summary:
        print(f" {results_summary[rho]['RF/Base']:.4f} & {results_summary[rho]['XGB/Base']:.4f} & "
            f"{results_summary[rho]['Linear/Base']:.4f} & {results_summary[rho]['RF Optimal/Base']:.4f} & "
            f"{results_summary[rho]['XGB Optimal/Base']:.4f} & {results_summary[rho]['Linear Optimal/Base']:.4f} & "
            f"{results_summary[rho]['Optimal Pool/Base']:.4f} \\\\")


Running simulations for rho = 0 ...


Iterations:  10%|█         | 100/1000 [00:46<07:15,  2.07it/s]/Users/ilmun/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Iterations: 100%|██████████| 1000/1000 [08:14<00:00,  2.02it/s]


Results for rho = 0:
  Linear / Base:      4.0136
  RF / Base:         1.0296
  XGB / Base:        1.0714
  Linear Optimal / Base:0.9999
  RF Optimal / Base: 1.0011
  XGB Optimal / Base:1.0011
  Optimal Pool / Base:1.0011
\begin{table}[h]
\centering
\begin{tabular}{cccccccc}
\toprule
 $\rho$ & Linear/Base & RF/Base & XGB/Base & Linear Optimal/Base & RF Optimal/Base & XGB Optimal/Base & Optimal Pool/Base \\
\midrule
 0 & 4.0136 & 1.0296 & 1.0714 & 0.9999 & 1.0011 & 1.0011 & 1.0011 \\
 4.0136 & 1.0296 & 1.0714 & 0.9999 & 1.0011 & 1.0011 & 1.0011 \\

Running simulations for rho = 0.25 ...


Iterations: 100%|██████████| 1000/1000 [08:36<00:00,  1.94it/s]


Results for rho = 0.25:
  Linear / Base:      4.4611
  RF / Base:         0.9703
  XGB / Base:        1.0082
  Linear Optimal / Base:1.0063
  RF Optimal / Base: 0.9706
  XGB Optimal / Base:0.9859
  Optimal Pool / Base:0.9725
\begin{table}[h]
\centering
\begin{tabular}{cccccccc}
\toprule
 $\rho$ & Linear/Base & RF/Base & XGB/Base & Linear Optimal/Base & RF Optimal/Base & XGB Optimal/Base & Optimal Pool/Base \\
\midrule
 0 & 4.0136 & 1.0296 & 1.0714 & 0.9999 & 1.0011 & 1.0011 & 1.0011 \\
 0.25 & 4.4611 & 0.9703 & 1.0082 & 1.0063 & 0.9706 & 0.9859 & 0.9725 \\
 4.0136 & 1.0296 & 1.0714 & 0.9999 & 1.0011 & 1.0011 & 1.0011 \\
 4.4611 & 0.9703 & 1.0082 & 1.0063 & 0.9706 & 0.9859 & 0.9725 \\

Running simulations for rho = 0.5 ...


Iterations: 100%|██████████| 1000/1000 [08:04<00:00,  2.06it/s]


Results for rho = 0.5:
  Linear / Base:      4.2522
  RF / Base:         0.7951
  XGB / Base:        0.7870
  Linear Optimal / Base:1.0134
  RF Optimal / Base: 0.7921
  XGB Optimal / Base:0.8189
  Optimal Pool / Base:0.7946
\begin{table}[h]
\centering
\begin{tabular}{cccccccc}
\toprule
 $\rho$ & Linear/Base & RF/Base & XGB/Base & Linear Optimal/Base & RF Optimal/Base & XGB Optimal/Base & Optimal Pool/Base \\
\midrule
 0 & 4.0136 & 1.0296 & 1.0714 & 0.9999 & 1.0011 & 1.0011 & 1.0011 \\
 0.25 & 4.4611 & 0.9703 & 1.0082 & 1.0063 & 0.9706 & 0.9859 & 0.9725 \\
 0.5 & 4.2522 & 0.7951 & 0.7870 & 1.0134 & 0.7921 & 0.8189 & 0.7946 \\
 4.0136 & 1.0296 & 1.0714 & 0.9999 & 1.0011 & 1.0011 & 1.0011 \\
 4.4611 & 0.9703 & 1.0082 & 1.0063 & 0.9706 & 0.9859 & 0.9725 \\
 4.2522 & 0.7951 & 0.7870 & 1.0134 & 0.7921 & 0.8189 & 0.7946 \\

Running simulations for rho = 0.9 ...


Iterations: 100%|██████████| 1000/1000 [06:11<00:00,  2.69it/s]


Results for rho = 0.9:
  Linear / Base:      2.9714
  RF / Base:         0.4024
  XGB / Base:        0.4029
  Linear Optimal / Base:1.0187
  RF Optimal / Base: 0.3866
  XGB Optimal / Base:0.4128
  Optimal Pool / Base:0.3915
\begin{table}[h]
\centering
\begin{tabular}{cccccccc}
\toprule
 $\rho$ & Linear/Base & RF/Base & XGB/Base & Linear Optimal/Base & RF Optimal/Base & XGB Optimal/Base & Optimal Pool/Base \\
\midrule
 0 & 4.0136 & 1.0296 & 1.0714 & 0.9999 & 1.0011 & 1.0011 & 1.0011 \\
 0.25 & 4.4611 & 0.9703 & 1.0082 & 1.0063 & 0.9706 & 0.9859 & 0.9725 \\
 0.5 & 4.2522 & 0.7951 & 0.7870 & 1.0134 & 0.7921 & 0.8189 & 0.7946 \\
 0.9 & 2.9714 & 0.4024 & 0.4029 & 1.0187 & 0.3866 & 0.4128 & 0.3915 \\
 4.0136 & 1.0296 & 1.0714 & 0.9999 & 1.0011 & 1.0011 & 1.0011 \\
 4.4611 & 0.9703 & 1.0082 & 1.0063 & 0.9706 & 0.9859 & 0.9725 \\
 4.2522 & 0.7951 & 0.7870 & 1.0134 & 0.7921 & 0.8189 & 0.7946 \\
 2.9714 & 0.4024 & 0.4029 & 1.0187 & 0.3866 & 0.4128 & 0.3915 \\
